**Лабораторная работа №2. Реализация глубокой нейронной сети**
1.   Реализуйте полносвязную нейронную сеть с помощью библиотеки Tensor Flow. В качестве алгоритма оптимизации можно использовать, например, стохастический градиент (Stochastic Gradient Descent, SGD). Определите количество скрытых слоев от 1 до 5, количество нейронов в каждом из слоев до нескольких сотен, а также их функции активации (кусочно-линейная, сигмоидная, гиперболический тангенс и т.д.).
2.   Как улучшилась точность классификатора по сравнению с логистической регрессией?
3. Используйте регуляризацию и метод сброса нейронов (dropout) для борьбы с переобучением. Как улучшилось качество классификации?
4. Воспользуйтесь динамически изменяемой скоростью обучения (learning rate). Наилучшая точность, достигнутая с помощью данной модели составляет 97.1%. Какую точность демонстрирует Ваша реализованная модель?

In [0]:
import os
try:
  import wget
except: 
  !pip install wget
  import wget
import tarfile


out_dir = 'data/not_mnist'
small_arhive = f'{out_dir}/notMNIST_small.tar.gz'
large_arhive = f'{out_dir}/notMNIST_large.tar.gz'
large_url = 'https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz'
small_url = 'https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz'

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=50e0f3f3d45a0ce8bfeec8caf1f9529fa273e431678a0908ad1e120e091ed412
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
if not os.path.exists(out_dir):
  os.makedirs(out_dir)

if not os.path.exists(small_arhive):
  print(f"Downloading {small_arhive}.")
  wget.download(small_url, small_arhive)
  print()
else:
  print(f"Skipping {small_arhive} download (already exists)")

if not os.path.exists(large_arhive):
  print(f"Downloading {large_arhive}.")
  wget.download(large_url, large_arhive)
  print()
else:
  print(f"Skipping {large_arhive} download (already exists)")

In [0]:
print(f"Extracting {small_arhive}")
with tarfile.open(small_arhive) as tar:
  tar.extractall(out_dir)

print(f"Extracting {large_arhive}")
with tarfile.open(large_arhive) as tar:
  tar.extractall(out_dir)

Extracting data/not_mnist/notMNIST_small.tar.gz
Extracting data/not_mnist/notMNIST_large.tar.gz


In [0]:
import numpy as np
from pathlib import Path
from PIL import Image

def remove_duplicates(img_train, labels_train, img_test):
    img_new, labels_new = [], []
    test_set = {e.tostring() for e in img_test}
    for i, (x, y) in enumerate(zip(img_train, labels_train)):
        if x.tostring() not in test_set:
            img_new.append(x)
            labels_new.append(y)

    print(f'Removed {img_train.shape[0] - len(img_new)} duplicated images')
    return np.array(img_new), np.array(labels_new)

def load_images(path, n):
    labels = ['I', 'G', 'A', 'F', 'H', 'J', 'C', 'D', 'E', 'B']

    x, y = [], []
    for i, l in enumerate(labels):
        d = Path(path) / l
        print(f'Loading {str(d)} ', end='')
        for j, f in zip(range(n), d.iterdir()):
            try:
                with Image.open(f) as img:
                    x.append(np.array(img))
                    y.append(i)
            except OSError:
                pass
            if j % 1000 == 0:
                print('.', end='', flush=True)
        print(flush=True)
    return np.array(labels), np.array(x), np.array(y)

def load_not_mnist_data(path='data/not_mnist/', use_cache=True):
    train_folder = Path(path) / 'notMNIST_large'
    test_folder = Path(path) / 'notMNIST_small'

    train_cache_file = Path(path) / 'train.npz'
    test_cache_file = Path(path) / 'test.npz'

    if train_cache_file.exists() and test_cache_file.exists() and use_cache:
        f = np.load(train_cache_file)
        labels, img_train, labels_train = [v for k, v in f.items()]
        f = np.load(test_cache_file)
        labels, img_test, labels_test = [v for k, v in f.items()]
        print('Loaded cached arrays')

    else:
        labels, img_train, labels_train = load_images(train_folder, 10000000)
        labels, img_test, labels_test = load_images(test_folder, 10000000)
        np.savez(train_cache_file, labels, img_train, labels_train)
        np.savez(test_cache_file, labels, img_test, labels_test)

    return labels, img_train, labels_train, img_test, labels_test

In [0]:
labels, img_train, labels_train, img_test, labels_test = load_not_mnist_data()

Loaded cached arrays


In [0]:
img_train, labels_train = remove_duplicates(img_train, labels_train, img_test)

Removed 12213 duplicated images


In [0]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split

def flatten(a):
    return a.reshape(a.shape[0], a.shape[1] * a.shape[2])

def load_data():
    labels, img_train, labels_train, img_test, labels_test = load_not_mnist_data()
    img_train, labels_train = remove_duplicates(img_train, labels_train, img_test)
    return labels, flatten(img_train), labels_train, flatten(img_test), labels_test

In [0]:

labels, img_train, labels_train, img_test, labels_test = load_not_mnist_data()
img_train, labels_train = remove_duplicates(img_train, labels_train, img_test)
    

Loaded cached arrays
Removed 12213 duplicated images


**Задание 1.**
Реализуйте полносвязную нейронную сеть с помощью библиотеки Tensor Flow. В качестве алгоритма оптимизации можно использовать, например, стохастический градиент (Stochastic Gradient Descent, SGD). Определите количество скрытых слоев от 1 до 5, количество нейронов в каждом из слоев до нескольких сотен, а также их функции активации (кусочно-линейная, сигмоидная, гиперболический тангенс и т.д.).


In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.callbacks import EarlyStopping

results = {}
results.setdefault('val_acc', {})

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
model.fit(x=img_train, y=labels_train, epochs=100,
          callbacks=[callback], validation_split=0.1)
print('\n# Evaluate')
result = model.evaluate(img_test, labels_test)
print(result)


Epoch 1/100
16154/16154 [==============================] - 44s 3ms/step - loss: 0.9557 - accuracy: 0.7816
Epoch 2/100
16154/16154 [==============================] - 43s 3ms/step - loss: 0.5695 - accuracy: 0.8335
Epoch 3/100
16154/16154 [==============================] - 43s 3ms/step - loss: 0.5467 - accuracy: 0.8385
Epoch 4/100
16154/16154 [==============================] - 42s 3ms/step - loss: 0.5205 - accuracy: 0.8465
Epoch 5/100
16154/16154 [==============================] - 44s 3ms/step - loss: 0.5058 - accuracy: 0.8506
Epoch 6/100
16154/16154 [==============================] - 43s 3ms/step - loss: 0.4956 - accuracy: 0.8537
Epoch 7/100
16154/16154 [==============================] - 43s 3ms/step - loss: 0.4864 - accuracy: 0.8555
Epoch 8/100
16154/16154 [==============================] - 43s 3ms/step - loss: 0.4766 - accuracy: 0.8586
Epoch 9/100
16154/16154 [==============================] - 43s 3ms/step - loss: 0.4686 - accuracy: 0.8604
Epoch 10/100
16154/16154 [====================

**Задание 2.**
Как улучшилась точность классификатора по сравнению с логистической регрессией?

Точность классификатора повысилась на 11 процентов

**Задание 3.**
Используйте регуляризацию и метод сброса нейронов (dropout) для борьбы с переобучением. Как улучшилось качество классификации?

В моей модели качество классификации не улучшилось при добавлении, скорее всего штраф был большим, и отбрасывались веса, которые могли бы описать удачнее модель, пытался уменьшать штраф, так как результат был еще меньше чем без регуляризации


In [0]:
initializer = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None)
regL1 = tf.keras.regularizers.l1(0.0001)
regL2 = tf.keras.regularizers.l2(0.0001)
model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(200, 
                          activation='relu', 
                          kernel_regularizer=regL2,
                          bias_regularizer=regL2, 
                          bias_initializer=initializer,
                          kernel_initializer=initializer),
    tf.keras.layers.Dense(200, 
                          activation='relu', 
                          kernel_regularizer=regL2,
                          bias_regularizer=regL2, 
                          bias_initializer=initializer,
                          kernel_initializer=initializer),
    tf.keras.layers.Dense(10, activation='softmax')
])

model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#steps_per_epoch=tf.math.ceil(train_size/BATCH_SIZE).numpy()
#model.fit(train_dataset, epochs=10, steps_per_epoch=steps_per_epoch, validation_data=(test_dataset))
model2.fit(x=img_train, y=labels_train, epochs=100,
          callbacks=[callback], validation_split=0.1)
#model.fit(train_dataset, epochs=2, steps_per_epoch=steps_per_epoch)
print('\n# Evaluate')
result2 = model2.evaluate(img_test, labels_test)
print (result2)

Epoch 1/100
16154/16154 [==============================] - 50s 3ms/step - loss: 0.7922 - accuracy: 0.8100
Epoch 2/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.5592 - accuracy: 0.8408
Epoch 3/100
16154/16154 [==============================] - 50s 3ms/step - loss: 0.5473 - accuracy: 0.8456
Epoch 4/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.5333 - accuracy: 0.8509
Epoch 5/100
16154/16154 [==============================] - 50s 3ms/step - loss: 0.5266 - accuracy: 0.8529
Epoch 6/100
16154/16154 [==============================] - 48s 3ms/step - loss: 0.5244 - accuracy: 0.8544
Epoch 7/100
16154/16154 [==============================] - 48s 3ms/step - loss: 0.5219 - accuracy: 0.8550
Epoch 8/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.5200 - accuracy: 0.8562
Epoch 9/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.5197 - accuracy: 0.8566
Epoch 10/100
16154/16154 [====================

In [0]:

model3 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(200, 
                          activation='relu', 
                          kernel_regularizer=regL2,
                          bias_regularizer=regL2, 
                          bias_initializer=initializer,
                          kernel_initializer=initializer),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(200, 
                          activation='relu', 
                          kernel_regularizer=regL2,
                          bias_regularizer=regL2, 
                          bias_initializer=initializer,
                          kernel_initializer=initializer),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(10, activation='softmax')
])

model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#steps_per_epoch=tf.math.ceil(train_size/BATCH_SIZE).numpy()
#model.fit(train_dataset, epochs=10, steps_per_epoch=steps_per_epoch, validation_data=(test_dataset))
model3.fit(x=img_train, y=labels_train, epochs=100,
          callbacks=[callback], validation_split=0.1)
#model.fit(train_dataset, epochs=2, steps_per_epoch=steps_per_epoch)
print('\n# Evaluate')
result = model3.evaluate(img_test, labels_test)
print (result)

Epoch 1/100
16154/16154 [==============================] - 50s 3ms/step - loss: 1.0940 - accuracy: 0.7254
Epoch 2/100
16154/16154 [==============================] - 50s 3ms/step - loss: 0.6626 - accuracy: 0.8030
Epoch 3/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.6378 - accuracy: 0.8119
Epoch 4/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.6360 - accuracy: 0.8107
Epoch 5/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.6263 - accuracy: 0.8154
Epoch 6/100
16154/16154 [==============================] - 50s 3ms/step - loss: 0.6150 - accuracy: 0.8178
Epoch 7/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.6083 - accuracy: 0.8191
Epoch 8/100
16154/16154 [==============================] - 48s 3ms/step - loss: 0.6029 - accuracy: 0.8211
Epoch 9/100
16154/16154 [==============================] - 49s 3ms/step - loss: 0.5996 - accuracy: 0.8218
Epoch 10/100
16154/16154 [====================

**Задание 4.**
Воспользуйтесь динамически изменяемой скоростью обучения (learning rate). Наилучшая точность, достигнутая с помощью данной модели составляет 97.1%. Какую точность демонстрирует Ваша реализованная модель?

Сравнение алгоритма без динамического шага, все предыдущие были с оптимизатором adam


In [0]:
model4 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
learning_rate = 0.001
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate)
model4.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#steps_per_epoch=tf.math.ceil(train_size/BATCH_SIZE).numpy()
#model.fit(train_dataset, epochs=10, steps_per_epoch=steps_per_epoch, validation_data=(test_dataset))
model4.fit(x=img_train, y=labels_train, epochs=100,
          callbacks=[callback], validation_split=0.1)
#model.fit(train_dataset, epochs=2, steps_per_epoch=steps_per_epoch)
print('\n# Evaluate')
result = model4.evaluate(img_test, labels_test)
print (result)

Epoch 1/100
14538/14538 [==============================] - 76s 5ms/step - loss: 0.7099 - accuracy: 0.8281 - val_loss: 3.9638 - val_accuracy: 0.3058
Epoch 2/100
14538/14538 [==============================] - 76s 5ms/step - loss: 0.4403 - accuracy: 0.8622 - val_loss: 3.0196 - val_accuracy: 0.4446
Epoch 3/100
14538/14538 [==============================] - 74s 5ms/step - loss: 0.4049 - accuracy: 0.8737 - val_loss: 3.0639 - val_accuracy: 0.4615
Epoch 4/100
14538/14538 [==============================] - 74s 5ms/step - loss: 0.3809 - accuracy: 0.8807 - val_loss: 2.5569 - val_accuracy: 0.5482
Epoch 5/100
14538/14538 [==============================] - 78s 5ms/step - loss: 0.3631 - accuracy: 0.8860 - val_loss: 2.8790 - val_accuracy: 0.5354
Epoch 6/100
14538/14538 [==============================] - 76s 5ms/step - loss: 0.3487 - accuracy: 0.8904 - val_loss: 3.0503 - val_accuracy: 0.4918
Epoch 7/100
14538/14538 [==============================] - 73s 5ms/step - loss: 0.3365 - accuracy: 0.8938 - val_